# 기본 세팅

In [1]:
# 모듈 import
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
import pandas as pd
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

# 페이지 맨 아래까지 스크롤 다운하는 함수
def scroll_down():
    # 끝까지 스크롤 다운
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 페이지 로드 대기
        time.sleep(2)  # 페이지 로딩 대기 시간 조정 가능

        # 새로운 높이 계산
        new_height = driver.execute_script("return document.body.scrollHeight")

        # 더 이상 스크롤할 내용이 없으면 종료
        if new_height == last_height:
            break
        last_height = new_height

# 텍스트 찾기 함수
def find_all_text(x):
    if x:
        x_texts = [i.text.strip() for i in x]
        x_text = ', '.join(x_texts)
    else:
        x_text = "Not found"
    
    return x_text

# 메뉴 클릭 함수
def click_menu(menu):
    # 탭 클릭
    try:
    # CSS 선택자를 사용하여 'veBoZ' 클래스를 가진 <span> 요소 중 텍스트가 menu인 요소 찾기
        menu_buttons = driver.find_elements(By.CSS_SELECTOR, 'a._tab-menu span.veBoZ')

        # '메뉴' 텍스트를 가진 요소 클릭
        for button in menu_buttons:
            if menu in button.text:
                button.click()
                break
    except Exception as e:
        print(f"오류 발생: {e}")
        
def scroll_up():
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)  # 스크롤 후 페이지 로딩 대기
    
def add_query_param_to_url(url, param_name, param_value):
    """현재 URL에 쿼리 파라미터를 추가하여 반환"""
    url_parts = list(urlparse(url))
    query = dict(parse_qs(url_parts[4]))
    query[param_name] = param_value
    url_parts[4] = urlencode(query, doseq=True)
    return urlunparse(url_parts)

# 식당 홈 탭 크롤링
- 식당 이름
- 메뉴 카테고리
- 주소
- 전화번호
- 웹사이트 주소
- 영업 시간
- 역에서부터의 거리
- 서비스 목록
- 총 리뷰 수

In [2]:
def home_page_data():
    time.sleep(5)
    scroll_down()

    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # 데이터 쌓기
    # 가게 이름
    try:
        restaurant_name = soup.find('span', class_='GHAhO').text.strip()
    except:
        restaurant_name = float('nan')
        
    # 업종
    try:
        category = soup.find('span', class_='lnJFt').text.strip()
    except:
        category = float('nan')
        
    # 총 방문자 리뷰 수
    try:
        count_reviews = soup.find_all('em', class_='place_section_count')
        second_count_review = count_reviews[1].text.strip()
    except:
        second_count_review = float('nan')
        
    # 주소
    try:
        address = soup.find('div', class_='O8qbU tQY7D').find('span', class_='LDgIH').text.strip()
    except:
        address = float('nan')
       
    # 역 기준 거리
    try:
        distance = soup.find('div', class_='nZapA').text.strip()
    except:
        distance = float('nan')

    # 영업 시간
    try:
        business_hours = soup.find('div', class_='O8qbU pSavy').find('span', class_='A_cdD').text.strip()
    except:
        business_hours = float('nan')

    # 전화번호
    try:
        phone_number = soup.find('div', class_='O8qbU nbXkr').find('span', class_='xlx7Q').text.strip()
    except:
        phone_number = float('nan')
        
    # 홈페이지
    try:
        # 'div' 태그에서 class가 'O8qbU yIPfO'인 요소를 찾고, 그 안에 있는 'a' 태그를 모두 찾기
        website_links = soup.find('div', class_='O8qbU yIPfO').find_all('a', class_='place_bluelink')
        # website_links_hrefs = [link.get('href') for link in website_links]
    except:
        website_links = float('nan')
        
    # 부가 서비스 목록
    try:
        home_service = soup.find('div', class_='xPvPE').text.strip()
    except:
        home_service = float('nan')

    return restaurant_name, category, second_count_review, address, distance, business_hours, phone_number, website_links, home_service

# 메뉴 탭 크롤링
- 메뉴 텍스트 정보

In [3]:
def menu_page_data():
    #메뉴 탭 클릭
    click_menu('메뉴')
    
    time.sleep(5)
    scroll_down()
    
    # 메뉴 더보기 버튼 클릭
    while True:
        try:
            more_menu_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div[6]/div/div[1]/div[2]/div/a')
            if more_menu_button.is_displayed() and more_menu_button.is_enabled():
                more_menu_button.click()
                time.sleep(2)  # 클릭 후 로딩 대기
            else:
                break
        except NoSuchElementException:
            break
    
    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(page_source, 'html.parser')

    # 메뉴 텍스트 내용 긁어오기
    try:
        menu_texts = soup.find_all('div', class_='MXkFw')
        if not menu_texts:  # 만약 찾은 결과가 비어있다면
            raise ValueError("First selector not found")
        menu_contents = [menu_text.get_text(separator='//').strip() for menu_text in menu_texts]
    except (AttributeError, ValueError):
        try:
            menu_texts = soup.find_all('div', class_='info_detail')
            if not menu_texts:  # 만약 찾은 결과가 비어있다면
                raise ValueError("Second selector not found")
            menu_contents = [menu_text.get_text(separator='//').strip() for menu_text in menu_texts]
        except (AttributeError, ValueError):
            menu_contents = float('nan')
        
    # 결과 반환
    return menu_contents

# 정보 탭 크롤링
- 소개글
- 편의시설 및 서비스
- 주차
- 좌석, 공간

In [22]:
import re

def info_page_data(ID):
#     # 정보 탭 클릭
#     click_menu('정보')

    # 정보 링크로 이동
    driver.get(f'https://m.place.naver.com/restaurant/{ID}/information')

    time.sleep(5)
    scroll_down()

    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # 데이터 쌓기
    # 소개글
    #     try:
    #         introduce_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div[5]/div/div[1]/div/div/div[2]/a')
    #         introduce_button.click()
    #         introduce = soup.find('div', class_='T8RFa').text.strip()
    #     except:
    #         introduce = float('nan')
    
    # 편의 시설 및 서비스
    try:
        service = soup.find_all('div', class_='owG4q')
        services = find_all_text(service)
        
    except:
        services = float('nan')
    
    # 주차
    try:
        parking = soup.find('div', class_='TZ6eS').text.strip()
    except:
        parking = float('nan')
    
    # 좌석, 공간
    try:
        seat = soup.find_all('li', class_='Lw5L1')
        seats = find_all_text(seat)
        
    except:
        seats = float('nan')
        
    # 소개
    try:
        info = soup.find('div', class_=re.compile(r'^T8RFa')).text
        
    except:
        info = float('nan')
    
    return services, parking, seats, info

# 리뷰 탭 크롤링
- 리뷰 텍스트 내용

In [10]:
def review_page_data():
    # 리뷰 탭 클릭
    click_menu('리뷰')
    time.sleep(5)
    
    # scroll_up()
    current_url = driver.current_url
    updated_url = add_query_param_to_url(current_url, 'reviewSort', 'recent')
    driver.get(updated_url)
    time.sleep(5)  # 페이지 로딩 대기
    
    # 키워드 리뷰 데이터 쌓기
    while True:
        try:
            # "더보기" 버튼의 XPath
            more_review_keyword_xpath = '/html/body/div[3]/div/div/div/div[6]/div[2]/div[1]/div/div/div[2]/a[1]'

            # "더보기" 버튼 요소 찾기
            more_review_keyword = driver.find_element(By.XPATH, more_review_keyword_xpath)

            # "더보기" 버튼 클릭
            more_review_keyword.click()

            # 클릭 후 페이지 로딩 대기
            time.sleep(2)

            # 'a' 태그의 'dP0sq' 클래스를 가진 요소 찾기
            dP0sq_elements = driver.find_elements(By.CLASS_NAME, 'dP0sq')

            if not dP0sq_elements:
                # 'dP0sq' 클래스를 가진 요소가 없으면 루프 종료
                break

        except NoSuchElementException:
            # "더보기" 버튼을 찾을 수 없으면 루프 종료
            break

    scroll_down()

    # 리뷰 키워드 정보 추출
    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(page_source, 'html.parser')

    # 모든 't3JSf' 태그와 'CUoLy' 태그를 찾음
    review_keywords_elements = soup.find_all('span', class_='t3JSf')
    review_keywords_count_elements = soup.find_all('span', class_='CUoLy')

    # 두 리스트의 길이를 확인하여 최소 길이만큼 순회
    min_length = min(len(review_keywords_elements), len(review_keywords_count_elements))

    # 키워드 데이터 리스트 생성
    review_keywords_data = []

    # 순서대로 함께 추가
    for i in range(min_length):
        review_keywords = review_keywords_elements[i].text.strip()
        review_keywords_count = review_keywords_count_elements[i].text.strip()
        review_keywords_data.append({
            'keyword': review_keywords,
            'count': review_keywords_count
        })
    
    scroll_down()
    
    # 끝까지 스크롤 다운 및 더보기 클릭 반복
    i = 0
    while i < 9:
        # 끝까지 스크롤 다운
        last_height = driver.execute_script("return document.body.scrollHeight")

        max_scroll_attempts = 1  # 최대 스크롤 시도 횟수 설정
        scroll_attempts = 0

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # 페이지 로딩 대기 시간 조정 가능
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scroll_attempts += 1
                if scroll_attempts >= max_scroll_attempts:
                    break
            else:
                scroll_attempts = 0  # 높이가 변경되면 시도 횟수 초기화
            last_height = new_height
        
        # 더보기 버튼 클릭
        try:
            more_review_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a')
            # 더보기 버튼이 있는 곳으로 스크롤
            driver.execute_script("arguments[0].scrollIntoView(true);", more_review_button)
            time.sleep(1)  # 스크롤 후 로딩 대기
            if more_review_button.is_displayed() and more_review_button.is_enabled():
                more_review_button.click()
                i += 1
                time.sleep(2)  # 클릭 후 로딩 대기
                
        except ElementClickInterceptedException:
            # 스크롤을 조금씩 위로 올리면서 클릭을 시도
            height = -100
            while True:
                driver.execute_script(f"window.scrollBy(0, {height});")
                time.sleep(1)  # 스크롤 후 로딩 대기
                try:
                    more_review_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a')
                    if more_review_button.is_displayed() and more_review_button.is_enabled():
                        more_review_button.click()
                        i += 1
                        time.sleep(2)  # 클릭 후 로딩 대기
                        break  # 클릭 성공 시 루프 탈출
                except ElementClickInterceptedException:
                    height -= 100  # height를 -100씩 더 줄임
                except:
                    break
        except NoSuchElementException:
            break
        except ElementNotInteractableException:
            break
            
            
    # 개별 리뷰 더보기 버튼
    # 초기 XPath
    base_xpath = '/html/body/div[3]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[{}]/div/div[4]/a'

    # 버튼을 순차적으로 클릭
    button_index = 1
    while True:
        try:
            # 현재 XPath
            current_xpath = base_xpath.format(button_index)

            # 해당 XPath에 해당하는 요소가 있는지 확인
            button = driver.find_element(By.XPATH, current_xpath)

            # 요소가 있다면 클릭하고 인덱스를 증가시킴
            button.click()
            time.sleep(2)  # 클릭 후 페이지 로딩 대기
            button_index += 1
        except:
            break
            
    scroll_up()
        
    while True:
        try:
            # CSS 선택자를 사용하여 'sIv5s WPk67' 클래스를 가진 <a> 태그 찾기
            review_buttons = driver.find_elements(By.CSS_SELECTOR, 'a.sIv5s.WPk67[role="button"]')

            if not review_buttons:
                # 더 이상 클릭할 버튼이 없으면 종료
                break

            for button in review_buttons:
                try:
                    button.click()
                    time.sleep(2)  # 클릭 후 페이지 로딩 대기
                except ElementNotInteractableException:
                    continue
        except NoSuchElementException:
            print("리뷰 더보기 버튼을 찾을 수 없습니다.")
            break
        except Exception as e:
            print(f"오류 발생: {e}")
            break
            
    scroll_down()

    # 페이지 로드 후 HTML 소스를 BeautifulSoup로 파싱
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    
    try:
        review_texts = soup.find_all('span', class_='zPfVt')
        keyword_review_texts = soup.find_all('div', class_='ERkm0')
    except:
        review_texts = float('nan')
        keyword_review_texts = float('nan')

    # 텍스트 추출
    review_text_list = [span.get_text() for span in review_texts]
    keyword_review_texts_list = [div.get_text(separator = ',') for div in keyword_review_texts]
    
    return review_keywords_data, review_text_list, keyword_review_texts_list

### 크롤링 실행


In [23]:
# 데이터프레임 로드
# dong = pd.read_csv('/Users/jeonjoeun/Desktop/naver_res_신수동.csv')

target = '창천동'
dong = pd.read_csv(f'./{target}.csv')


# Chrome WebDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# 결과 저장을 위한 빈 리스트 생성
results = []

# 각 ID에 대해 데이터를 수집하고 리스트에 저장
for index, row in dong[:10].iterrows():
    ID = row['ID']
    url = f'https://m.place.naver.com/restaurant/{ID}/home?entry=pll'
    driver.get(url)
    
    home_data = home_page_data()
    menu_data = menu_page_data()
    info_data = info_page_data(ID)
    review_data = review_page_data()
    print(info_data)
    print(f'100 중 {index} 완')
    
    # 데이터를 하나의 튜플로 결합
    combined_data = home_data + (menu_data,) + info_data + review_data
    results.append(combined_data)

# 결과를 데이터프레임으로 변환
columns = [
    'restaurant_name', 'category', 'second_count_review', 'address', 'distance', 
    'business_hours', 'phone_number', 'website_links', 'home_service', 'menu_contents', 
    'services', 'parking', 'seats', 'info','review_keywords_data', 'review_text_list', 'keyword_review_texts_list'
]
results_df = pd.DataFrame(results, columns=columns)

# 기존 데이터프레임과 결합
final_df = pd.concat([dong, results_df], axis=1)

# 데이터프레임 저장
final_df.to_csv(f'./{target}_상세정보.csv', index=False)


('단체 이용 가능', nan, 'Not found', '1988년에 오픈한 수제비 전문점입니다. 진한 사골 국물에, 볶은 호박과 당근, 다진 고기를 고명으로 올려낸 사골 수제비와 밥을 원하시는 분들을 위한 김밥, 두가지 메뉴를 판매합니다. 식사 시간에는 줄을 서서 기다리는 손님이 많으니 참고하세요.')
100 중 0 완
('포장, 무선 인터넷, 남/녀 화장실 구분, 예약', '주차불가', 'Not found', '신촌에서 신촌현대백화점 연세대학교와 함께 벌써\n14년째 신촌과 함께 시간을 보내며 매장을 운영해왔습니다.\n연세대학교 서강대학교 이화여자대학교 홍익대학교\n어학연수로 어학당에서 공부를 하셨던 외국인 학생들까지\n정말 많은 분들과 추억을 만들어가고 있습니다.\n\n연세세브란스 병원에서 치료받고 휠체어를 타고 찾아주셨던\n많은 고객님들도 진심으로 감사드립니다.\n크레페라는 생소한 디저트를 가지고 한국에서 가장 맛있는\n크레페를 만들어보겠다는 진심하나로 힘든시간도 있었지만\n그동안 찾아주신 고객분들에 감사한마음으로 지금까지\n신촌에서 매장을 운영중입니다.\n\n신촌역 1번 출구로 운동 겸 1분만 걸으면 신촌에서 가장 유명한 디저트맛집 커피크레페입니다. 네이버 길찾기 내비게이션 준비 당 충전 하실 준비되셨죠?\n신촌 현대백화점 또는 신촌 유플렉스에서 쇼핑 잘하셨나요?신촌CGV에서 인기영화도 보시고 혹시 건강의 문제가 있어서 연세세브란스 병원에서 치료받아셨다면 기운도 없으시고 당충전 필요하시면 당충전은 저희가 전문입니다. 인스타 네이버 사진촬영 1위\n신촌 벚꽂축제 버스킹 구경하시고 출출하시면 신촌 커피크레페 잊지말고 찾아주시길 바랍니다. 아이와 함께 데이트 사랑하는 연인 친구 맛있는 디저트 먹고 사진찍기 가족과함께하는 특별한 디저트를 먹으면서 신촌에서 풍경을 감상하실 수 있는 커피 버블티 아침에 간단하게 드실수 있는 식사용디저트 점심에서 식사후 드실수 있는 상큼한 과일디저트 저녁에 마무리로 아이스크림이 들어간 달콤한 디저트로 하루를 마무리 해보실는걸 추천

In [18]:
final_df

,Unnamed: 0,지번주소,도로명주소,업체명,업종,URL,ID,restaurant_name,category,second_count_review,...,website_links,home_service,menu_contents,services,parking,seats,info,review_keywords_data,review_text_list,keyword_review_texts_list
0,0,서울특별시 서대문구 창천동 30-10번지,서울특별시 서대문구 신촌로 87-8 (창천동),신촌수제비,한식,https://pcmap.place.naver.com/restaurant/19878...,19878436,철길부산집 신촌점,요리주점,NaN,...,[[https://www.instagram.com/busanjip_sinchon]],"무선 인터넷, 포장","[어묵 2인//대표//17,000//원, 숙성연어 사시미//대표//29,000//원...",단체 이용 가능,NaN,Not found,"1988년에 오픈한 수제비 전문점입니다. 진한 사골 국물에, 볶은 호박과 당근, 다...","[{'keyword': '""음식이 맛있어요""', 'count': '이 키워드를 선택...",[신촌역 이자카야맛집입니다~!! 특히 야끼소바 정말 미쳤어여 ㅜ 꼭드세요 하이볼도 ...,"[인테리어가 멋져요,음식이 맛있어요,술이 다양해요,가성비가 좋아요, 음악이 좋아요,..."
1,1,서울특별시 서대문구 창천동 30-10번지,서울특별시 서대문구 신촌로 87-8 (창천동),크레페마스터,"카페,디저트",https://pcmap.place.naver.com/restaurant/19796...,19796689,철길부산집 신촌점,요리주점,NaN,...,[[https://www.instagram.com/busanjip_sinchon]],"무선 인터넷, 포장","[어묵 2인//대표//17,000//원, 숙성연어 사시미//대표//29,000//원...","포장, 무선 인터넷, 남/녀 화장실 구분, 예약",주차불가,Not found,신촌에서 신촌현대백화점 연세대학교와 함께 벌써\n14년째 신촌과 함께 시간을 보내며...,"[{'keyword': '""음식이 맛있어요""', 'count': '이 키워드를 선택...",[신촌역 이자카야맛집입니다~!! 특히 야끼소바 정말 미쳤어여 ㅜ 꼭드세요 하이볼도 ...,"[인테리어가 멋져요,음식이 맛있어요,술이 다양해요,가성비가 좋아요, 음악이 좋아요,..."
2,2,서울특별시 서대문구 창천동 30-10번지,서울특별시 서대문구 신촌로 87-8 (창천동),버거리 신촌점,햄버거,https://pcmap.place.naver.com/restaurant/11043...,1104306587,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,서울특별시 서대문구 창천동 13-26번지 1층,"서울특별시 서대문구 명물길 18, 1층 (창천동)",대구삼겹살,돼지고기구이,https://pcmap.place.naver.com/restaurant/31397...,31397200,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,서울특별시 서대문구 창천동 13-26번지 1층,"서울특별시 서대문구 명물길 18, 1층 (창천동)",미네르바,카페,https://pcmap.place.naver.com/restaurant/35562...,35562920,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,1,서울특별시 서대문구 창천동 33-43번지 (지하1층),"서울특별시 서대문구 연세로 23 (창천동,(지하1층))",채선당 신촌점,샤브샤브,https://pcmap.place.naver.com/restaurant/18746...,18746815,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
523,0,서울특별시 서대문구 창천동 13-9 광연빌딩,"서울특별시 서대문구 명물길 26, 광연빌딩 2층 (창천동)",포가레 신촌점,베트남음식,https://pcmap.place.naver.com/restaurant/10143...,1014319726,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524,1,서울특별시 서대문구 창천동 13-9 광연빌딩,"서울특별시 서대문구 명물길 26, 광연빌딩 2층 (창천동)",엠브로돈까스 신촌본점,돈가스,https://pcmap.place.naver.com/restaurant/18159...,1815955748,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
525,2,서울특별시 서대문구 창천동 13-9 광연빌딩,"서울특별시 서대문구 명물길 26, 광연빌딩 2층 (창천동)",홍콩반점0410 신촌2호점,중식당,https://pcmap.place.naver.com/restaurant/31044...,31044483,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
len(final_df.review_text_list[0])

100

In [25]:
final_df.review_text_list[2]

['제가 신촌에서 젤 좋아하는 카페예요 일단 자리가 편하고 요즘 트렌드 같은 예쁜 카페 느낌은 아니지만 아늑하고 특별한 메뉴가 있구!! ♥️ 핫초코 메뉴가 특별하게 당도 세 단계로 나눠져 있어요 이 카페를 좋아하는 게 제 개인적인 추억 때문도 있지만 아늑하고 편한 분위기를 좋아하는 분들은 분명 좋아하실 거 같아요',
 '신촌 유일무이 핫초코 맛집이에요\n원래는 얼죽아여도 여기선 웬만하면 핫초코만 먹게 되는 거 같아요~ 카페 특유의 포근한 분위기와 메뉴, 사장님 모든 게 다 잘 어울리는 좋은 카페',
 '카페 추천받아서 왔는데 분위기랑 좌석이 넘 좋았어요!! 사장님도 짱짱 친절하셔서 몇 마디 얘기 안 나눴는데 기분이 좋아졌어요ㅎㅎ 노래 선곡도 너무 취저입니다🫶',
 '카페도 넓고 카공하기 딱 좋은 팟입니당 \n시험기간에 항상 애용하고있어요😍😍',
 '레모네이드도 맛있었고 공부하기 좋았어요!',
 '',
 '신촌 최고의 카공 카페\n주문메뉴: 단맛없는 다크초코 추천!',
 '참 차분하고 편안한 곳이네요. 소곤소곤 얘기를 나누어도 나쁘지 않고 모두들 아주 조용합니다. 음료는 맛도 최고고 더하기 정성이 가득하게 담겨있네요. \n만남의 장소로도\n혼자 보내는 장소로도\n차 맛을 즐기는 곳으로도\n다~ 좋아요 ~^^',
 '가끔 카공하러 가는데 사장님께서 완전 친절하시고 커피도 맛있어요! 아이스 초코도 자주 먹는데 강추합니다~',
 '좌석도 편하고 조용해서 좋아요:)',
 '차 종류가 많아요 사장님 차에 진심인듯...!\n녹차 쉐이큰데 녹차향 가득하고 많이 안달아서 좋아요\n얼그레이 좋아하시는 분 얼그레이 밀크티 꼭 드세요\n입 안 가득 얼그레이 행복합니다',
 '라떼 jmt',
 '컵도 너무 귀엽고 사장님도 너무 친절하셔서 또 가고픈 카페입니다ㅠ 공부하기도 적절하고 도란도란 얘기나누기도 좋은 카페인것 같아요',
 '굿',
 '조용하고 좋아요 :)',
 '옛날 소개팅 장소로 선택했다면 백전백승이었을 것 같은 카페예요',
 '아지트같은 공간이에요.',
 '굿',
 '조용하